In [4]:
#使用TensorFlow 之前，首先导入它
import tensorflow as tf

In [5]:
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
sess = tf.InteractiveSession()

In [14]:
# 我们先从创建输入图像和输出类别的节点来创建计算图。
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

In [15]:
sess.run(tf.initialize_all_variables())

In [16]:
# 权重初始化

# 在创建模型之前，我们先来创建权重和偏置。一般来说，初始化时应加入轻微噪声，
# 来打破对称性，防止零梯度的问题。因为我们用的是ReLU，所以用稍大于0的值来初
# 始化偏置能够避免节点输出恒为0 的问题（dead neurons）。为了不在建立模型的时候
# 反复做初始化操作，我们定义两个函数用于初始化。
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [17]:
# 卷积和池化

# TensorFlow 在卷积和池化上有很强的灵活性。我们怎么处理边界？步长应该设多
# 大？在这个实例里，我们会一直使用vanilla 版本。我们的卷积使用1 步长（stride size），
# 0 边距（padding size）的模板，保证输出和输入是同一个大小。我们的池化用简单传统
# 的2*2 大小的模板做max pooling。为了代码更简洁，我们把这部分抽象成一个函数。
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [18]:
# 第一层卷积

# 现在我们可以开始实现第一层了。它由一个卷积接一个max pooling 完成。卷积在
# 每个5*5 的patch 中算出32 个特征。权重是一个[5, 5, 1, 32]的张量，前两个维度是
# patch 的大小，接着是输入的通道数目，最后是输出的通道数目。输出对应一个同样大
# 小的偏置向量。
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# 为了用这一层，我们把x变成一个4d 向量，第2、3 维对应图片的宽高，最后一维代
# 表颜色通道。
x_image = tf.reshape(x, [-1,28,28,1])

# 我们把x_image和权值向量进行卷积相乘，加上偏置，使用ReLU激活函数，最后max
# pooling。
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [20]:
# 第二层卷积

# 为了构建一个更深的网络，我们会把几个类似的层堆叠起来。第二层中，每个5x5
# 的patch 会得到64 个特征。
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [21]:
# 密集连接层

# 现在，图片降维到7*7，我们加入一个有1024 个神经元的全连接层，用于处理整
# 个图片。我们把池化层输出的张量reshape 成一些向量，乘上权重矩阵，加上偏置，使
# 用ReLU 激活。
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat , W_fc1) + b_fc1)

In [22]:
# Dropout

# 为了减少过拟合，我们在输出层之前加入dropout。我们用一个placeholder 来代表
# 一个神经元在dropout 中被保留的概率。这样我们可以在训练过程中启用dropout，在
# 测试过程中关闭dropout。TensorFlow 的tf.nn.dropout操作会自动处理神经元输出值的
# scale。所以用dropout 的时候可以不用考虑scale。
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [23]:
# 输出层

# 最后，我们添加一个softmax 层，就像前面的单层softmax regression 一样。
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [ ]:
# 训练和评估模型

# 这次效果又有多好呢？我们用前面几乎一样的代码来测测看。只是我们会用更加复
# 杂的ADAM 优化器来做梯度最速下降，在feed_dict 中加入额外的参数keep_prob 来控
# 制dropout 比例。然后每100 次迭代输出一次日志。
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.14
step 100, training accuracy 0.82
step 200, training accuracy 0.9
